In [ ]:
! git clone https://github.com/Min-KiD/video_features.git
! pip install omegaconf==2.0.6

In [ ]:
%cd video_features

In [ ]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path, form_list_from_user_input
from omegaconf import OmegaConf
from tqdm import tqdm
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

In [ ]:
import cv2
import math

def count_segments(video_path):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    segment_size = math.ceil(total_frames / 32)
    return segment_size

## Extract feature 1 video by put directory in list video_paths, set file_with_video_paths to None

## Extract feature videos in txt file by put directory in file txt to file_with_video_paths, set video_paths to None

In [ ]:
# Example usage with 1 video
video_paths = ['/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse002_x264.mp4'] # extract 1 video
file_with_video_paths = None

# Example usage with file txt contain many videos
# video_paths = None
# file_with_video_paths = '/kaggle/input/crimeucfdataset/normal.txt'

In [ ]:
feature_type = 'i3d'

video_paths = form_list_from_user_input(video_paths, file_with_video_paths, to_shuffle=True)

print(f'The number of specified videos: {len(video_paths)}')

for video_path in tqdm(video_paths):
    segment_size = count_segments(video_path)
    
    args = OmegaConf.load(build_cfg_path(feature_type))
    args.stack_size = segment_size
    args.step_size = segment_size
    args.flow_type = 'raft'
    args.on_extraction = 'save_numpy'
    
    extractor = ExtractI3D(args)
    extractor._extract(video_path)

# for video_path in args.video_paths:
#     print(f'Extracting for {video_path}')
#     feature_dict = extractor.extract(video_path)
#     [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]